<h2 style="text-align: center;">Segmentación de Clientes con Análisis RFM</h2>


In [6]:
# Importar librería
import pandas as pd
import os

In [7]:
# Leer y cargar el conjunto de datos
data = pd.read_parquet('../../datasets/online_retail.parquet')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2023-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2023-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2023-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2023-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2023-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
# Exploración inicial del DataFrame para revisar estructura, tipos de datos y valores nulos
print('Información general del conjunto de datos:\n')
data.info()

# Estadísticas descriptivas para entender la distribución de las variables
print('\nEstadísticas descriptivas de las columnas:')
data.describe(include='all')

Información general del conjunto de datos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  541909 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB

Estadísticas descriptivas de las columnas:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909,541909,541909,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900,4070,4224,NaN,NaN,NaN,NaN,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,1114,2313,2369,NaN,NaN,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,2024-07-04 08:55:02.927097344,4.611114,15287.690570,NaN
min,NaN,NaN,NaN,-80995.000000,2023-12-01 08:26:00,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2024-03-28 11:34:00,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,2024-07-19 17:17:00,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,2024-10-19 11:27:00,4.130000,16791.000000,NaN
max,NaN,NaN,NaN,80995.000000,2024-12-09 12:50:00,38970.000000,18287.000000,NaN


In [9]:
# Conteo de valores nulos por columna para evaluar la calidad del dataset
data.isna().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [10]:
# Crear una copia del DataFrame original para preservar los datos sin modificar
df = data.copy()

# Definir condiciones para filtrar transacciones válidas:
# - Excluir facturas que comienzan con letras (cancelaciones, etc.)
# - CustomerID no nulo
# - Cantidad positiva
# - Precio unitario positivo
mask = (
    ~(df['InvoiceNo'].str.match(r'^[A-Za-z]', na=False))  # Excluir facturas no estándar
    & (df['CustomerID'].notna())                          # Cliente identificado
    & (df['Quantity'] > 0)                                # Transacción positiva
    & (df['UnitPrice'] > 0)                               # Precio válido
)

# Aplicar el filtro
df = df.loc[mask]

# Eliminar registros duplicados y reiniciar el indice
df = (df.drop_duplicates(
        subset=['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate'])
        .reset_index(drop=True)
)

# Agregar columna de Ingreso
df['Revenue'] = df['Quantity'] * df['UnitPrice']

# Normalizar columna InvoiceDate
df['InvoiceDate'] = df['InvoiceDate'].dt.floor('D')

# Mostrar el DataFrame limpio
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
253204,568142,85150,LADIES & GENTLEMEN METAL SIGN,1,2024-09-25,2.55,16744.0,United Kingdom,2.55
101900,549696,22423,REGENCY CAKESTAND 3 TIER,12,2024-04-11,12.75,13089.0,United Kingdom,153.00
121943,552310,21123,SET/10 IVORY POLKADOT PARTY CANDLES,3,2024-05-08,1.25,17323.0,United Kingdom,3.75
146196,555516,23209,LUNCH BAG DOILEY PATTERN,4,2024-06-05,1.65,17706.0,United Kingdom,6.60
300981,572653,23266,SET OF 3 WOODEN STOCKING DECORATION,6,2024-10-25,1.25,15861.0,United Kingdom,7.50
14990,538255,37501,TEA TIME TEA SET IN GIFT BOX,3,2023-12-10,6.95,14911.0,EIRE,20.85
9369,537433,21754,HOME BUILDING BLOCK WORD,18,2023-12-06,5.45,12913.0,United Kingdom,98.10
206714,562980,23034,DRAWER KNOB CERAMIC BLACK,6,2024-08-11,1.45,17686.0,United Kingdom,8.70
58758,544409,22084,PAPER CHAIN KIT EMPIRE,12,2024-02-18,2.95,16265.0,United Kingdom,35.40
119080,551998,21872,GLAMOROUS MUG,6,2024-05-05,1.25,14099.0,United Kingdom,7.50


In [11]:
# Análisis RFM (Recency, Frequency, Monetary)

# Obtener la fecha de la última compra registrada
last_purchase_date = df['InvoiceDate'].max()

# Agrupar por cliente y calcular:
# - Recency: días desde la última compra
# - Frequency: número de facturas únicas
# - Monetary: ingreso total generado
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (last_purchase_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                        # Frequency
    'Revenue': 'sum'                                               # Monetary
}).rename(
    columns={
        'InvoiceDate': 'Recency',
        'InvoiceNo': 'Frequency',
        'Revenue': 'Monetary'
    }
).reset_index()

rfm.sample(10)


,CustomerID,Recency,Frequency,Monetary
2315,15503.0,363,1,141.04
3302,16843.0,42,16,10196.57
141,12522.0,39,2,192.72
3772,17512.0,28,10,2924.99
683,13253.0,156,2,1030.40
2951,16368.0,2,2,479.91
1395,14240.0,30,2,890.34
710,13292.0,38,2,578.94
307,12723.0,3,5,725.07
3618,17301.0,21,1,163.24


In [12]:
# Calcular puntuaciones RFM

rfm['R_Score'] = pd.qcut(rfm['Recency'], q=3, labels=[3, 2, 1])  # Menor recency es mejor
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=3, labels=[1, 2, 3])  # Mayor frecuencia es mejor
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=3, labels=[1, 2, 3])  # Mayor gasto es mejor

# Combinar las puntuaciones en un solo score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

rfm.sample(10)  # Tabla con RFM Score

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
4176,18069.0,26,5,2009.24,2,3,3,233
1125,13870.0,2,8,2770.94,3,3,3,333
969,13652.0,45,3,1095.08,2,2,2,222
3506,17134.0,106,2,413.20,1,2,2,122
4301,18233.0,326,1,440.00,1,2,2,122
2505,15753.0,305,1,79.20,1,1,1,111
1224,14006.0,115,3,962.19,1,2,2,122
1821,14830.0,53,1,188.88,2,1,1,211
3333,16892.0,1,5,518.19,3,3,2,332
12,12359.0,57,4,6310.03,2,2,3,223


In [13]:
# Clasificación de clientes según RFM

def assing_segement(r, f, m):
    if r == 3 and f == 3 and m == 3:
        return 'VIP Customer'
    elif r == 3 and f >= 2 and m >= 2:
        return 'Loyal Customer'
    elif r == 3:
        return 'New Customer'
    elif r == 2 and f == 3:
        return 'Potential Loyallist'
    elif f == 1 and m == 1:
        return 'At Risk'
    elif r == 1:
        return 'Lost Customer'
    else:
        return 'Promising Customer'

# Aplicar clasificación por fila
rfm['Segment'] = rfm.apply(lambda row: assing_segement(
    int(row['R_Score']),
    int(row['F_Score']),
    int(row['M_Score'])),
    axis=1
)

rfm  # Tabla final con segmentos

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346.0,326,1,77183.60,1,1,3,113,Lost Customer
1,12347.0,2,7,4310.00,3,3,3,333,VIP Customer
2,12348.0,75,4,1797.24,2,2,3,223,Promising Customer
3,12349.0,18,1,1757.55,3,1,3,313,New Customer
4,12350.0,311,1,334.40,1,1,1,111,At Risk
...,...,...,...,...,...,...,...,...,...
4333,18280.0,277,1,180.60,1,2,1,121,Lost Customer
4334,18281.0,180,1,80.82,1,2,1,121,Lost Customer
4335,18282.0,7,2,178.05,3,2,1,321,New Customer
4336,18283.0,3,16,2045.53,3,3,3,333,VIP Customer


In [14]:
# Exportar archivos del análisis
#rfm.to_csv('../../datasets/rfm_analysis.csv', sep=',', index=False)
#rfm.to_excel('../../datasets/rfm_analysis.xlsx', sheet_name='data', index=False)